# Preparation

In [1]:
from Echo_cancellation_utils import EchoCancellation
%matplotlib widget

# Load the class
EC = EchoCancellation()

# Exercise

We have received the following audio file:

In [2]:
EC.display_audio_file()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

As can be heard, the audio has some undesirable echoes. The aim of this exercise is to remove the echo and to obtain a clean signal. The audio file is stored numerically in the signal $y[n]$ whose size is $N=185140$ and has a sampling frequency $F_s=48000\;Hz$.

* Q: Determince the length $T$ of the signal **in seconds** (up to two digits)?

In [3]:
EC.Q1

We consider the following model for echo:

$$y[n]=x[n]+ax[n-n_0]\;\;\;\;\;\;\;\;\;\;(1)$$
where
* $x[n]$ is the clean signal
* $a \in {\rm I\!R}$ with $|a|\lt 1$ is the attenuation factor
* $n_0 \in \mathbb{N}$ is the shift

Let us recall that for a discrete signal $x[n]$, its auto correlation function is defined as

$$\mathrm{R_x}[m]=\sum_{n \in \mathbb{Z}} x[n]x[m+n]$$

Using $(1)$, we can compute the auto correlation function of $y[n]$, denoted by $\mathrm{R_y}[m]$, as 

$$\mathrm{R_y}[m] = a_0\mathrm{R_x}[m] + a_1\mathrm{R_x}[m-m_1] + a_2\mathrm{R_x}[m-m_2],$$

where $a_0$, $a_1$, $a_2$ $\in {\rm I\!R}$ and $n_1$, $n_2$ $\in \mathbb{Z}$ depend on the model parameters $(a, n_0)$.

* Q: For the special case $n_0=5$ and $a=\frac{1}{2}$, give the values of $a_0$, $a_1$, $a_2$ $\in {\rm I\!R}$ as well as $m_1$, $m_2$ $\in \mathbb{Z}$ assuming the convention $m_1 \lt m_2$.

In [4]:
EC.Q2

To determine the unknown parameters $(a, n_0)$, we rely on the assumption that $\mathrm{R_x}[m]$ has only one major peak at $m=0$ and it rapidly decreases as $m$ grows (see the following figure for an illustration).

In [5]:
EC.display_auto_corr_x()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Using this assumption, we can claim that $\mathrm{R_y}[m]$ has a main peak at $m=0$ with height $a_0\mathrm{R_x}[0]$ and two side peaks at $m=m_1$ and $m=m_2$ with heights $a_1\mathrm{R_x}[0]$ and $a_2\mathrm{R_x}[0]$, respectively.

Now let us look at the auto correlation function $\mathrm{R_y}[m]$.

In [6]:
EC.display_auto_corr_y()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

* Q: Using this plot and the above description, estimate the unknown parameters $a \in {\rm I\!R}$ and $n_0 \in \mathbb{N}$.

In [7]:
EC.Q3

* Q: Using $n_0$ and the sample frequency $F_s$, determine the echo time $T_e$ **in seconds**.

In [8]:
EC.Q4

Based on your answer, we now apply the following inverse filter to $y[n]$:

$$y[n] \longrightarrow \boxed{\frac{1}{1 + az^{-n_0}}} \longrightarrow \tilde{x}[n]$$

Let's hear $\tilde{x}[n]$:

In [10]:
EC.display_x_filtered()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<div class='alert alert-success'>
    <b>If you hear a lie, then Congratulations!</b>
<div>